Basierend auf unserer fachlichen Expertise und der explorativen Datenanalyse (EDA) könnten folgende Variablen relevant sein, um den Preis einer Immobilie vorherzusagen:

1. Standortbezogene Variablen
   - distanceToCenter: Entfernung zum Zentrum
   - northeast, northwest, southeast, southwest: Spezifische Standorte

2. Grundlegende Eigenschaften
   - constructionYear: Baujahr des Objekts
   - livingSpace: Wohnfläche in Quadratmetern (auch log_livingspace wird berücksichtigt)
   
3. Energie und Nachhaltigkeit
   - energyClass: Energieeffizienzklasse
   - energySources: Energieträger des Objekts **(entfernt - statisch nicht signifikant)**

4. Komfort und Ausstattung
   - elevator: Vorhandensein eines Aufzugs
   - sauna: Vorhandensein einer Sauna
   - swimmingpool: Vorhandensein eines Swimmingpools
   - kitchenEquipped: Ausgestattete Küche
   - cellar: Vorhandensein eines Kellers
   - storageRoom: Abstellraum

5. Zugänglichkeit und zusätzliche Merkmale
   - wheelchairAccessible: Barrierefreiheit
   - assistedLiving: Betreutes Wohnen

6. Außenbereich und Extras
   - parkingSlots: Parkplätze
   - privateOutdoorSpace: Privater Außenbereich
   - chimney: Kamin
   - cuestToilet: Gäste-WC
   - attics: Wohnung im Dachgeschoss

7. Zielvariable
   - price und log_price: Preis des Objekts (Predictor)

In [1]:
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.formula.api as smf
import ast

import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns  

# seaborn settings
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

import altair as alt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
filePath = r'https://raw.githubusercontent.com/hdm-statistik-lpj/project/main/data/external/immobilienNuernberg_cleaned.csv'
df = pd.read_csv(filePath)

In [3]:
df.columns

Index(['location', 'Northeast', 'Northwest', 'Southeast', 'Southwest',
       'energyEfficiency', 'price', 'constructionYear', 'livingSpace',
       'energySources', 'photosCount', 'localAmenitiesScore',
       'localMobilityScore', 'latitudeDistrict', 'longitudeDistrict',
       'distanceToCenter', 'district', 'rooms', 'energyClass', 'elevator',
       'sauna', 'swimmingpool', 'kitchenEqipped', 'cellar', 'storageRoom',
       'wheelchairAccessible', 'assistedLiving', 'parkingSlots', 'guestToilet',
       'privateOutdoorSpace', 'chimney', 'attics'],
      dtype='object')

In [4]:
# Nummerische Variablen
df['constructionYear'] = df['constructionYear'].astype(int)
df['livingSpace'] = df['livingSpace'].astype(float)
df['photosCount'] = df['photosCount'].astype(int)
df['localAmenitiesScore'] = df['localAmenitiesScore'].astype(float)
df['localMobilityScore'] = df['localMobilityScore'].astype(float)
df['latitudeDistrict'] = df['latitudeDistrict'].astype(float)
df['longitudeDistrict'] = df['longitudeDistrict'].astype(float)
df['distanceToCenter'] = df['distanceToCenter'].astype(float)
df['price'] = df['price'].astype(float)
df['rooms'] = df['rooms'].astype(float)

# Kategorische Variablen
df['district'] = df['district'].astype('category')
df['location'] = df['location'].astype('category')
df['energyClass'] = df['energyClass'].astype('category')
df['energySources'] = df['energySources'].astype('category')

# Binäre Variablen
binaryColumns = ['elevator', 'sauna', 'swimmingpool', 'kitchenEqipped', 
                  'cellar', 'storageRoom', 'wheelchairAccessible', 'assistedLiving', 
                  'parkingSlots', 'guestToilet', 'privateOutdoorSpace', 
                  'chimney', 'attics', 'energyEfficiency', 
                  'Northeast', 'Northwest', 'Southeast', 'Southwest']

for col in binaryColumns:
    df[col] = df[col].astype('category')

In [5]:
# Modell 1: 'price' als abhängige Variable
X_price = df.drop('price', axis=1)
y_price = df['price']
X_price['log_livingSpace'] = np.log(X_price['livingSpace'])

In [6]:
# Aufteilen der Daten für das Modell mit 'price' als abhängige Variable
X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(
    X_price, y_price, test_size=0.2, random_state=42, stratify=df['energyClass'])
df_train_price = pd.concat([X_train_price, y_train_price], axis=1)
df_test_price = pd.concat([X_test_price, y_test_price], axis=1)

In [7]:
import statsmodels.formula.api as smf

# formula 
formula = 'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + attics'

# Fit 
model = smf.ols(formula=formula, data=df_train_price).fit()

# summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(price)   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.844
Method:                 Least Squares   F-statistic:                     140.5
Date:                Tue, 26 Dec 2023   Prob (F-statistic):          5.10e-180
Time:                        16:08:46   Log-Likelihood:                 117.25
No. Observations:                 490   AIC:                            -194.5
Df Residuals:                     470   BIC:                            -110.6
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               4.5715    

attics[T.1]: Der Koeffizient ist 0.0681 und der p-Wert ist 0.083.

#### Die Variable attics wird im nächsten Modell nicht berücksichtigt